## Question 1


In [1]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
# opens the csv and converts the figures that are strings into float

def open_file(workfile):
    L=[]
    f=open(workfile, "r")
    for line in f:
        l= [x for x in line.split(",")]
        L.append(l)
    lenght=len(L[0])
    for i in range(1, len(L)):
        for j in range(1,lenght):
            L[i][j]=float(L[i][j])
    return L

In [3]:
workfile="./data.csv"

In [4]:
# create an array ready to be used in the kmeans algorithm

def required_list_for_clustering(L):
    l=[] 
    for i in range(1,len(L)):
        l2=[j for j in L[i]]
        del l2[0]                
        l.append(l2)
    return np.asarray(l)

In [5]:
def kmeans(L,k,ini='random',max_=1000,n_in=100):
    kmeans=KMeans(init=ini,n_clusters=k,max_iter=max_,n_init=n_in).fit(L)
    return kmeans.predict(L), kmeans.cluster_centers_

In [6]:
# Our material for k-means :

L=required_list_for_clustering(open_file(workfile))

In [7]:
# function SSE, which calculates the square of the distances to the centroids of the cluster for each 
#point and do the sum

def SSE(L,k,ini='random',max_=1000,n_in=100):
    predict, center=kmeans(L,k,ini,max_,n_in)
    s=0
    for i in range(k):
        for j in range(len(predict)):
            if predict[j]==i:
                for m in range(len(L[j])):
                    s+=(L[j][m]-center[i][m])**2
    return s
                    
    


In [8]:
# First test :

SSE(L,8)

1590.816835111399

## Question 2 : Playing with the parameters to decrease SSE

#### First, I think that we can decrase the SSE by choosing better our initial centers, ie by usink k-means ++ instead of k-mean.

It is not necessary that it's going to improve the SSE

In [9]:
SSE(L,8,'k-means++')

1544.5883338858732

My prevision was true : the SSE is inferior to the former result, because our initialization was better. (1529<1578)

#### On the other hand, we can also play with the number of iteration. 
For now, the maximum number of iteration is 1000.

We can think that by doing more iteration, our result will become more accurate. But in fact, it's not always the case.

In [10]:
SSE(L,8,'random',max_=1000)

1600.5496443758714

In [11]:
SSE(L,8,'random',max_=10000)

1613.0267347735933

In [12]:
SSE(L,8,'random',max_=100000)

1621.019341052785

The result are very different at the end of each test, which proves that it do not have a huge impact on SSE. 
We keep 10000 which is on the average the better.

#### Finally, we can play with the number n_init. If we test more possibilities for initila centroids, we can wonder that the algorithm will run more efficiently, but it is not necessary. 

We can compare the efficiency of n_in by increasing it steadily : 

In [13]:
n_init=5
best_n_init=5
res=SSE(L,8,'k-means++',max_=10000,n_in=n_init)
for i in range(10):
    n_init=n_init*2
    a=SSE(L,8,'k-means++',max_=10000,n_in=n_init)
    if a<res:
        res=a
        best_n_init=n_init
        
best_n_init, res

(80, 1509.4291397575112)

We do not obtain the same value for each test.
That proves that it don't decrease necessarily the SSE. 
But in general, we can notice that it is preferable to have a huge value for the number of initialisations tested in order to have a low SSE, what is logical.

#### RESULT : With the parameters "k-means++", a number max of iterations equals to 10000, and a n-in equals to 5120, I obtained an SSE of 1508, better than the 1560 obtained at the beginning.

## Question 3 : 

We keep the parameters obtained at question 2.

In [14]:
def cluster(workfile,k=8,n_init=5120,max_iter=10000,ini='k-means++'):
    res=[]
    L=required_list_for_clustering(workfile)
    predict,cluster_center=kmeans(L,k,ini,max_iter,n_init)
    for i in range(k):
        L=[]
        for j in range(len(predict)):
            if predict[j]==i:
                L.append(workfile[j+1][0])
        res.append(L)
    return res

In [15]:
workfile="./data.csv"
workfile=open_file(workfile)
cluster=cluster(workfile)
cluster


[['American Express', 'Boeing', 'Microsoft', 'Walt Disney', 'JPMorgan Chase'],
 ['IBM',
  'The Home Depot',
  'Intel',
  'Wal-Mart',
  'General Electric',
  'United Technologies',
  'Travelers',
  '3M'],
 ['Chevron', 'Pfizer', 'ExxonMobil'],
 ['Kraft',
  'Verizon',
  'Procter & Gamble',
  'AT&T',
  'Merck',
  'McDonalds',
  'Coca-Cola',
  'Johnson & Johnson'],
 ['Hewlett-Packard'],
 ['DuPont', 'Caterpillar', 'Alcoa'],
 ['Cisco Systems'],
 ['Bank of America']]

First cluster :  JUNK FOOD & TELECOM

['Kraft','Verizon','Procter & Gamble','AT&T','Merck','McDonalds','Coca-Cola','Johnson & Johnson']

McDO and Coca are often in the same cluster.
We have also telecoms, with AT&T and Verizon.




Second cluster : INFORMATIC
    
['IBM','The Home Depot','Intel','Wal-Mart','General Electric','United Technologies','Travelers','3M']

Third cluster : BANK 
    
['American Express', 'Boeing', 'Microsoft', 'Walt Disney', 'JPMorgan Chase']

Fourth : HP, unfortunately it is not in the second

Fifth : Cisco, idem...

Sixth : ENERGY 
    
['Chevron', 'Pfizer', 'ExxonMobil']

Seventh: Bank of America, unfortunately it is not in the third

Eighth : Divers